# Рекомендательные системы

## Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

## Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

In [77]:
import numpy as np
from collections import Counter

На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

In [64]:
def freq_file(file):
    viewed_count = Counter()
    buyed_count = Counter()
    
    with open(file, 'rt') as f:
        for line in f.readlines():
            viewed, buyed = line.strip().split(';')
                
            viewed = viewed.split(',')
            buyed = buyed.split(',')
            
            for item in viewed:
                viewed_count[item] += 1
            for item in buyed:
                buyed_count[item] += 1
            
    return viewed_count, buyed_count

In [65]:
def read_file(file):   
    data = []
    
    with open(file, 'rt') as f:
        for line in f.readlines():
            viewed, buyed = line.strip().split(';')  
            
            if len(buyed) == 0:
                continue
                
            viewed = viewed.split(',')
            buyed = buyed.split(',')
            
            data.append((viewed,buyed))
            
    return data

In [66]:
viewed_count, buyed_count = freq_file('coursera_sessions_train.txt')

In [67]:
train_data = read_file('coursera_sessions_train.txt')
test_data = read_file('coursera_sessions_test.txt')

Реализуйте два алгоритма рекомендаций:
- сортировка просмотренных id по популярности (частота появления в просмотренных),
- сортировка просмотренных id по покупаемости (частота появления в покупках).

In [78]:
def predict(viewed, k, method='popular'):
    unique_viewed = np.array(viewed)[np.sort(np.unique(viewed, return_index=True)[1])]
    k = min(len(viewed), k)    
    ratings = []
    
    for item in unique_viewed:
        if method == 'popular':
            ratings.append(-viewed_count[item])
        else:
            ratings.append(-buyed_count[item])
    
    sorted_items = np.argsort(ratings)
    return list(unique_viewed[sorted_items])[0:k]

Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [79]:
def precision(predicted, buyed, k):
    rb = [x for x in buyed if x in predicted]
    return len(rb)/float(k)

In [80]:
def recall(predicted, buyed):
    rb = [x for x in buyed if x in predicted]
    return len(rb)/float(len(buyed))

In [87]:
def get_stats(data, method):
    ar1 = np.mean([recall(predict(user[0], 1, method=method), user[1]) for user in data])
    ap1 = np.mean([precision(predict(user[0], 1, method=method), user[1], 1) for user in data])
    ar5 = np.mean([recall(predict(user[0], 5, method=method), user[1]) for user in data])
    ap5 = np.mean([precision(predict(user[0], 5, method=method), user[1], 5) for user in data])
    
    return ar1, ap1, ar5, ap5

In [88]:
train_popular = get_stats(train_data, 'popular')
print(train_popular)
test_popular = get_stats(test_data, 'popular')
print(test_popular)
train_purch = get_stats(train_data, 'purch')
print(train_purch)
test_purch = get_stats(test_data, 'purch')
print(test_purch)

(0.44241489678712514, 0.511640798226164, 0.8245162888663443, 0.21247228381374728)
(0.4170598107364819, 0.4810368349249659, 0.8002413034096623, 0.2039836289222374)
(0.6888975707711849, 0.8048780487804879, 0.926246040852471, 0.2523835920177384)
(0.46017437579852316, 0.5268758526603001, 0.8165208320469471, 0.20878581173260574)
